# QuSO Algorithm

This notebook contains the full implementation of the QuSO algorithm

In [11]:
import numpy as np
import os, sys
from scipy.optimize import curve_fit

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)
from QuSO_utils import *

In [2]:
T_env = 293       # Ambient temperature (K)
R_env = 0.001      # Convection resistance to ambient (K/W)

# Heat Flows (in Watts)
# Positive values indicate heat generation; negative values indicate cooling.
Q_1 = 2000    
Q_2 = 4000    
Q_3 = -200    
Q_4 = -2000   

# Inter-node Thermal Resistances (in K/W)
# These values lump together conduction and convection effects.
R_12 = 0.005
R_13 = 0.006
R_14 = 0.006
R_23 = 0.007
R_24 = 0.007
R_34 = 0.008

R_dict = {
    (0, 1): R_12,
    (0, 2): R_13,
    (0, 3): R_14,
    (1, 2): R_23,
    (1, 3): R_24,
    (2, 3): R_34
}

connections = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

conductance_coeffs = [1/R_12, 1/R_13, 1/R_14, 1/R_23, 1/R_24, 1/R_34, 1/(2*R_env), 0]
C_l = np.sum(conductance_coeffs)**(-1/2)
conductance_coeffs_amps = np.sqrt(conductance_coeffs)*C_l

B = np.array([Q_1, Q_2, Q_3, Q_4])
C_B = np.sum([el**2 for el in B])**(-1/2)
B_amps = C_B*B

In [5]:
# define polynomial for QSVT
kappa = 3
pcoefs, C_p = pyqsp.poly.PolyOneOverX().generate(kappa, return_coef=True, ensure_bounded=True, return_scale=True, epsilon=0.01)
phi_pyqsp = QuantumSignalProcessingPhases(pcoefs, signal_operator="Wx")

b=51, j0=22
[PolyOneOverX] minimum [-4.58579384] is at [-0.15726745]: normalizing
[PolyOneOverX] bounding to 0.9


In [21]:
# define quantum registers
num_phase_wires = 2

config_wires = [f"config({i})" for i in range(len(connections))]
phase_wires = [f"p({i})" for i in range(num_phase_wires)]
q_aux = ["q"]
l_aux = [f"l({i})" for i in range(int(np.log2(len(conductance_coeffs_amps))))]
flag_aux = ["flag"]
l_prime_aux = ["l'"]
data_wires = [f"d({i})" for i in range(2)]

wires = config_wires + phase_wires + q_aux + l_aux + flag_aux + l_prime_aux + data_wires
dev = qml.device("lightning.qubit", wires=wires)

In [22]:
# functions for cost readout

def qpe_probability_function(y, theta, M):
    y = np.array(y, dtype=np.float64)
    numerator = 1 - np.cos(2 * np.pi * (y - theta * M))
    denominator = 1 - np.cos((2 * np.pi / M) * (y - theta * M))
    # When denominator is close to zero, use the limiting value (which gives a ratio of M^2).
    ratio = np.where(np.abs(denominator) < 1e-8, M**2, numerator / denominator)
    P = ratio / (M**2)
    return P

def qae_probability_function(y, theta, M):
    term1 = qpe_probability_function(y, theta, M)
    term2 = qpe_probability_function(y, -theta, M)
    return 0.5 * (term1 + term2)

def estimate_theta_qae(probabilities):
    probabilities = np.array(probabilities, dtype=np.float64)
    M = len(probabilities)  # Number of states (M = 2^n).
    y_vals = np.arange(M)
    
    max_index = np.argmax(probabilities)
    initial_guess = (max_index / M)
    
    # Define narrow bounds around the initial guess.
    lower_bound = max(0, initial_guess - 0.5 / M)
    upper_bound = min(1, initial_guess + 0.5 / M)
    bounds = ([lower_bound], [upper_bound])
    
    # Define the model function with theta as the only free parameter.
    def model(y, theta):
        return qae_probability_function(y, theta, M)
    
    # Attempt two fits (starting at the lower and upper bounds) to help avoid local minima.
    try:
        popt1, pcov1 = curve_fit(model, y_vals, probabilities, p0=[lower_bound], bounds=bounds)
    except Exception as e:
        popt1, pcov1 = (np.array([np.nan]), np.array([[np.inf]]))
        
    try:
        popt2, pcov2 = curve_fit(model, y_vals, probabilities, p0=[upper_bound], bounds=bounds)
    except Exception as e:
        popt2, pcov2 = (np.array([np.nan]), np.array([[np.inf]]))
    
    var1 = pcov1[0, 0] if np.isfinite(pcov1[0, 0]) else np.inf
    var2 = pcov2[0, 0] if np.isfinite(pcov2[0, 0]) else np.inf
    
    if var1 < var2:
        theta_est = popt1[0]
    else:
        theta_est = popt2[0]
    
    return theta_est

In [23]:
# define QAOA layers
def mixer_layer(beta, wires):
    for wire in wires:
        qml.RX(-2*beta, wires=wire)

sin_hamiltonian = get_func_hamiltonian(qae_func, -1, phase_wires)

def qaoa_layer(gamma, beta):
    cost_layer(gamma, sin_hamiltonian, phase_wires, [0, 0], B_amps, connections, conductance_coeffs_amps, phi_pyqsp, q_aux[0], config_wires, l_aux, flag_aux[0], l_prime_aux[0], data_wires)
    mixer_layer(beta, config_wires)

def qaoa_circuit(params, wires, depth):
    for wire in wires:
        qml.Hadamard(wires=wire)
    qml.layer(qaoa_layer, depth, params[0], params[1])

# number of QAOA layers
p = 1

@qml.qnode(dev)
def cost_function_circuit(params):
    qaoa_circuit(params, config_wires, p)
    QAE(phase_wires, [0, 0], B_amps, connections, conductance_coeffs_amps, phi_pyqsp, q_aux[0], config_wires, l_aux, flag_aux[0], l_prime_aux[0], data_wires)
    
    return qml.probs(wires=config_wires+phase_wires)

def cost_function(params):
    probs = cost_function_circuit(params)
    cost_curvefit = 0
    for i in range(2**len(config_wires)):
        probs_i = np.sum(probs.reshape((2**len(config_wires), -1)), axis=1)[i]
        probs_qae_i = probs.reshape((2**len(config_wires), -1))[i, :]/probs_i
        a = np.sin(np.pi*estimate_theta_qae(probs_qae_i))
        cost_curvefit += probs_i * a
    
    return cost_curvefit

Polynomial Expression: 1.73205080756888*x0*x1 - 0.866025403784439*x0 - 0.866025403784439*x1


In [26]:
cost_function([[0.5]*p, [0.5]*p])

/var/folders/by/tjkhpdvj401cj92bnwvg228h0000gn/T/ipykernel_59671/2402714845.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = np.where(np.abs(denominator) < 1e-8, M**2, numerator / denominator)


0.19799705582641758

In principle, the circuit functions correctly. However, as its size grows, it becomes increasingly difficult to simulate due to the required number of phase qubits and the complexity of the associated polynomial. Optimizing such large circuits poses an even greater challenge. 
on, we chose not to pursue that direction and instead analyzed QuSO using a simplified QAE approach based on state preparation. For further details, refer to `QuSO_adjusted.ipynb` and the accompanying paper.
